In [9]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [10]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid')) #softmax


In [11]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #categorical_crossentropy


In [12]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 21, 21, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [13]:
from keras.preprocessing.image import ImageDataGenerator


In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
training_set = train_datagen.flow_from_directory('../input/skin-cancer-malignant-vs-benign/train',
                                                 target_size = (64, 64), # make sure to change target_size if you altered input_shape above
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_set = test_datagen.flow_from_directory('../input/skin-cancer-malignant-vs-benign/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [21]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 64,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 256)

Epoch 1/25
64/64 [==============================] - 9s 147ms/step - loss: 0.4574 - accuracy: 0.7753 - val_loss: 0.4573 - val_accuracy: 0.7909
Epoch 2/25
64/64 [==============================] - 7s 116ms/step - loss: 0.4267 - accuracy: 0.7881
Epoch 3/25
64/64 [==============================] - 7s 107ms/step - loss: 0.4251 - accuracy: 0.7896
Epoch 4/25
64/64 [==============================] - 7s 111ms/step - loss: 0.4220 - accuracy: 0.7988
Epoch 5/25
64/64 [==============================] - 9s 133ms/step - loss: 0.4170 - accuracy: 0.7999
Epoch 6/25
64/64 [==============================] - 7s 115ms/step - loss: 0.4202 - accuracy: 0.7851
Epoch 7/25
64/64 [==============================] - 7s 109ms/step - loss: 0.3970 - accuracy: 0.8091
Epoch 8/25
64/64 [==============================] - 7s 114ms/step - loss: 0.4017 - accuracy: 0.8013
Epoch 9/25
64/64 [==============================] - 8s 119ms/step - loss: 0.4069 - accuracy: 0.8038
Epoch 10/25
64/64 [==============================] - 7s 11

In [22]:
# saving model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# saving weights to HDF5
classifier.save_weights("model.h5")

In [23]:
# Making prediction about user data
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator

In [35]:
# load json and create model
json_file = open('./model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./model.h5")

In [36]:
test_datagen = ImageDataGenerator(rescale = 1./255)
user_data = test_datagen.flow_from_directory('../input/skin-cancer-malignant-vs-benign/test',
                                            target_size = (64, 64),
                                            batch_size = 1,
                                            class_mode = 'binary')
 

Found 660 images belonging to 2 classes.


In [37]:
# evaluate loaded model on user data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(user_data)
if score[1] == 0:
    print('The convolutional neural network predicts that this tumor is malignant!')
else:
    print('The convolutional neural network predicts that this tumor is benign.')


660/660 [==============================] - 2s 4ms/step - loss: 0.3291 - accuracy: 0.8470
The convolutional neural network predicts that this tumor is benign.
